In [ ]:
!pip install -q transformers datasets seqeval
!pip install -q evaluate seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install -q transformers accelerate evaluate seqeval


In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from collections import defaultdict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Dataset.txt"
df = pd.read_csv(path)

df.head(40)

,Category,Text
0,Accountant,﻿________________\r\n\r\nEDUCATION\r\nOMBA - E...
1,Accountant,﻿________________\r\n\r\nHOWARD GERRARD\r\nAcc...
2,Accountant,﻿________________\r\n\r\nKevin Frank\r\nSENIOR...
3,Accountant,﻿________________\r\n\r\nPlace of birth\r\nNat...
4,Accountant,"﻿________________\r\n\r\nStephen Greet, CPA\r\..."
5,Accountant,﻿________________\r\n\r\nMarie Anderson\r\nSom...
6,Accountant,﻿________________\r\n\r\nCAROLE CHUN\r\n122 Ki...
7,Accountant,﻿________________\r\n\r\nAnna Gunther\r\nSenio...
8,Accountant,﻿________________\r\n\r\nFIRST LAST Cost Accou...
9,Accountant,﻿________________\r\n\r\nRobert Smith Associat...


In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13389 entries, 0 to 13388
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  13389 non-null  object
 1   Text      13389 non-null  object
dtypes: object(2)
memory usage: 209.3+ KB


In [ ]:
df['Text'].duplicated().sum()

np.int64(1214)

In [ ]:
df_unique = df.drop_duplicates(subset='Text').copy()

In [ ]:
df_unique['Text'].duplicated().sum()

np.int64(0)

In [ ]:
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12175 entries, 0 to 13388
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  12175 non-null  object
 1   Text      12175 non-null  object
dtypes: object(2)
memory usage: 285.4+ KB


In [ ]:
df_unique.value_counts('Category')

,count
Category,
Education,388
Electrical Engineering,363
Consultant,345
Sales,344
Digital Media,341
Mechanical Engineer,340
Accountant,337
Building and Construction,336
Operations Manager,332


In [ ]:
NER_LABELS = [
    "O",
    "B-NAME", "I-NAME",
    "B-EMAIL", "I-EMAIL",
    "B-PHONE", "I-PHONE",
    "B-ORG", "I-ORG",
    "B-JOB_TITLE", "I-JOB_TITLE",
    "B-DEGREE", "I-DEGREE",
    "B-DATE", "I-DATE",
    "B-SKILL", "I-SKILL"
]

label2id = {label: idx for idx, label in enumerate(NER_LABELS)}
id2label = {idx: label for label, idx in label2id.items()}

print(label2id)


{'O': 0, 'B-NAME': 1, 'I-NAME': 2, 'B-EMAIL': 3, 'I-EMAIL': 4, 'B-PHONE': 5, 'I-PHONE': 6, 'B-ORG': 7, 'I-ORG': 8, 'B-JOB_TITLE': 9, 'I-JOB_TITLE': 10, 'B-DEGREE': 11, 'I-DEGREE': 12, 'B-DATE': 13, 'I-DATE': 14, 'B-SKILL': 15, 'I-SKILL': 16}


In [ ]:
def simple_tokenize(text):
    """
    Basit whitespace + punctuation tokenizer
    (NER annotation için yeterli)
    """
    tokens = []
    for match in re.finditer(r'\S+', text):
        tokens.append({
            "token": match.group(),
            "start": match.start(),
            "end": match.end()
        })
    return tokens


In [ ]:
EMAIL_REGEX = re.compile(
    r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b'
)

PHONE_REGEX = re.compile(
    r'(\+?\d{1,3}[\s\-]?)?(\(?\d{3}\)?[\s\-]?)?\d{3}[\s\-]?\d{4}'
)

DATE_REGEX = re.compile(
    r'\b(?:'
    r'(?:\d{1,2}[./-]\d{1,2}[./-]\d{2,4})|'
    r'(?:\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s+\d{4})|'
    r'(?:\b\d{4}\b)'
    r')\b',
    re.IGNORECASE
)


In [ ]:
def spans_to_bio(tokens, spans, label_name):
    labels = ["O"] * len(tokens)

    for span_start, span_end in spans:
        first_token = True
        for i, tok in enumerate(tokens):
            if tok["end"] <= span_start or tok["start"] >= span_end:
                continue

            if first_token:
                labels[i] = f"B-{label_name}"
                first_token = False
            else:
                labels[i] = f"I-{label_name}"

    return labels


In [ ]:
sample_text = df_unique.iloc[0]["Text"]

tokens = simple_tokenize(sample_text)

email_spans = [(m.start(), m.end()) for m in EMAIL_REGEX.finditer(sample_text)]
phone_spans = [(m.start(), m.end()) for m in PHONE_REGEX.finditer(sample_text)]
date_spans  = [(m.start(), m.end()) for m in DATE_REGEX.finditer(sample_text)]

labels = ["O"] * len(tokens)

for spans, label in [
    (email_spans, "EMAIL"),
    (phone_spans, "PHONE"),
    (date_spans, "DATE")
]:
    new_labels = spans_to_bio(tokens, spans, label)
    labels = [
        nl if nl != "O" else l
        for l, nl in zip(labels, new_labels)
    ]

# Görsel kontrol
for t, l in zip(tokens[:40], labels[:40]):
    print(f"{t['token']:20s} {l}")


﻿________________    O
EDUCATION            O
OMBA                 O
-                    O
Executive            O
Leadership           O
University           O
of                   O
Texas                O
2016-2018            B-DATE
O                    O
Bachelor             O
of                   O
Science              O
in                   O
Accounting           O
Richland             O
College              O
2005-2008            B-DATE
TRAINING             O
&                    O
CERTIFICATIONS       O
Certified            O
Management           O
Accountant           O
(CMA)                O
Certified            O
Financial            O
Modeling             O
and                  O
Valuation            O
Analyst              O
Compliance           O
and                  O
Anti-Money           O
Laundering           O
(09/2016)            B-DATE
American             O
Institute            O
of                   O


In [ ]:
def annotate_text(text):
    tokens = simple_tokenize(text)

    email_spans = [(m.start(), m.end()) for m in EMAIL_REGEX.finditer(text)]
    phone_spans = [(m.start(), m.end()) for m in PHONE_REGEX.finditer(text)]
    date_spans  = [(m.start(), m.end()) for m in DATE_REGEX.finditer(text)]

    labels = ["O"] * len(tokens)

    for spans, label in [
        (email_spans, "EMAIL"),
        (phone_spans, "PHONE"),
        (date_spans, "DATE")
    ]:
        new_labels = spans_to_bio(tokens, spans, label)
        labels = [
            nl if nl != "O" else l
            for l, nl in zip(labels, new_labels)
        ]

    return [t["token"] for t in tokens], labels


In [ ]:
annotated_data = []

for idx, row in df_unique.iterrows():
    text = row["Text"]
    if not isinstance(text, str) or len(text.strip()) == 0:
        continue

    tokens, labels = annotate_text(text)

    if len(tokens) == len(labels) and len(tokens) > 0:
        annotated_data.append((tokens, labels))

print("Toplam CV sayısı:", len(annotated_data))


Toplam CV sayısı: 12175


In [ ]:
def write_conll(data, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        for tokens, labels in data:
            for t, l in zip(tokens, labels):
                f.write(f"{t}\t{l}\n")
            f.write("\n")

write_conll(annotated_data, "ner_weak_all.conll")


In [ ]:
SAVE_PATH = "/content/drive/MyDrive/nlp_cv_ner"


In [ ]:
import os
os.makedirs(SAVE_PATH, exist_ok=True)


In [ ]:
write_conll(
    annotated_data,
    os.path.join(SAVE_PATH, "ner_weak_all.conll")
)

In [ ]:
!ls /content/drive/MyDrive/nlp_cv_ner


ner_weak_all.conll


In [ ]:
def read_conll(path):
    sentences = []
    labels = []

    cur_tokens = []
    cur_labels = []

    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                if cur_tokens:
                    sentences.append(cur_tokens)
                    labels.append(cur_labels)
                cur_tokens = []
                cur_labels = []
            else:
                token, label = line.split("\t")
                cur_tokens.append(token)
                cur_labels.append(label)

    return sentences, labels


In [ ]:
DATA_PATH = "/content/drive/MyDrive/nlp_cv_ner/ner_weak_all.conll"

sentences, ner_labels = read_conll(DATA_PATH)

print("CV sayısı:", len(sentences))
print("Örnek token sayısı:", len(sentences[0]))
print("Örnek etiketler:", set(ner_labels[0]))


CV sayısı: 12175
Örnek token sayısı: 199
Örnek etiketler: {'B-DATE', 'O'}


In [ ]:
from sklearn.model_selection import train_test_split

train_tokens, val_tokens, train_labels, val_labels = train_test_split(
    sentences,
    ner_labels,
    test_size=0.1,
    random_state=42
)

print("Train:", len(train_tokens))
print("Validation:", len(val_tokens))


Train: 10957
Validation: 1218


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
label_list = NER_LABELS  # ADIM 1'de tanımlamıştık

label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}


In [ ]:
def tokenize_and_align_labels(tokens, labels):
    tokenized = tokenizer(
        tokens,
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=256
    )

    word_ids = tokenized.word_ids()
    aligned_labels = []

    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            aligned_labels.append(-100)  # ignore
        elif word_idx != previous_word_idx:
            aligned_labels.append(label2id[labels[word_idx]])
        else:
            # same word → I- label
            lab = labels[word_idx]
            if lab.startswith("B-"):
                lab = "I-" + lab[2:]
            aligned_labels.append(label2id.get(lab, label2id["O"]))
        previous_word_idx = word_idx

    tokenized["labels"] = aligned_labels
    return tokenized


In [ ]:
sample_tokens = train_tokens[0]
sample_labels = train_labels[0]

encoded = tokenize_and_align_labels(sample_tokens, sample_labels)

for t_id, l_id in zip(encoded["input_ids"][:30], encoded["labels"][:30]):
    tok = tokenizer.convert_ids_to_tokens(t_id)
    lab = id2label[l_id] if l_id != -100 else "IGN"
    print(f"{tok:15s} {lab}")


[CLS]           IGN
_               O
_               O
_               O
_               O
_               O
_               O
_               O
_               O
_               O
_               O
_               O
_               O
_               O
_               O
_               O
_               O
john            O
smith           O
22              O
street          O
rd              O
.               O
city            O
,               O
state           O
000             O
##00            O
phone           O
:               O


In [ ]:
from datasets import Dataset


In [ ]:
train_dataset = Dataset.from_dict({
    "tokens": train_tokens,
    "labels": train_labels
})

val_dataset = Dataset.from_dict({
    "tokens": val_tokens,
    "labels": val_labels
})


In [ ]:
train_dataset = train_dataset.map(
    lambda x: tokenize_and_align_labels(x["tokens"], x["labels"]),
    batched=False
)

val_dataset = val_dataset.map(
    lambda x: tokenize_and_align_labels(x["tokens"], x["labels"]),
    batched=False
)


Map:   0%|          | 0/10957 [00:00<?, ? examples/s]

Map:   0%|          | 0/1218 [00:00<?, ? examples/s]

In [ ]:
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)

val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)


In [ ]:
train_dataset[0]


{'labels': tensor([-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    5,    6,    6,   13,   14,   14,
           14,    0,    0,    3,    4,    4,    4,    4,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import evaluate

metric = evaluate.load("seqeval")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    true_labels = []
    true_predictions = []

    for pred, lab in zip(predictions, labels):
        t_lab = []
        t_pred = []
        for p, l in zip(pred, lab):
            if l != -100:
                t_lab.append(id2label[l])
                t_pred.append(id2label[p])
        true_labels.append(t_lab)
        true_predictions.append(t_pred)

    return metric.compute(
        predictions=true_predictions,
        references=true_labels
    )


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ner_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=100,
    report_to="none",
    load_best_model_at_end=True
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-890265614.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Date,Email,Phone,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.012300,0.012910,"{'precision': 0.841792656587473, 'recall': 0.8719239373601789, 'f1': 0.8565934065934067, 'number': 3576}","{'precision': 0.971689497716895, 'recall': 0.994392523364486, 'f1': 0.9829099307159354, 'number': 1070}","{'precision': 0.8622129436325678, 'recall': 0.9066959385290889, 'f1': 0.8838951310861424, 'number': 911}",0.869898,0.901206,0.885275,0.995598
2,0.008900,0.009400,"{'precision': 0.8585831062670299, 'recall': 0.881152125279642, 'f1': 0.8697212255037261, 'number': 3576}","{'precision': 0.9617486338797814, 'recall': 0.9869158878504672, 'f1': 0.9741697416974169, 'number': 1070}","{'precision': 0.9053763440860215, 'recall': 0.9242590559824369, 'f1': 0.9147202607278653, 'number': 911}",0.886100,0.908584,0.897201,0.996486
3,0.007800,0.008939,"{'precision': 0.855522549284364, 'recall': 0.8859060402684564, 'f1': 0.8704492375326282, 'number': 3576}","{'precision': 0.9733944954128441, 'recall': 0.991588785046729, 'f1': 0.9824074074074075, 'number': 1070}","{'precision': 0.9164859002169198, 'recall': 0.9275521405049396, 'f1': 0.9219858156028369, 'number': 911}",0.887839,0.913083,0.900284,0.996683


TrainOutput(global_step=4110, training_loss=0.019075919948790196, metrics={'train_runtime': 1662.6942, 'train_samples_per_second': 19.77, 'train_steps_per_second': 2.472, 'total_flos': 4295125245823488.0, 'train_loss': 0.019075919948790196, 'epoch': 3.0})

In [ ]:
import torch
from collections import defaultdict

def extract_entities_from_text(text, tokenizer, model, id2label, max_length=256):
    tokens = text.split()

    encoding = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        max_length=max_length
    )

    with torch.no_grad():
        outputs = model(**encoding)

    predictions = torch.argmax(outputs.logits, dim=-1)[0].tolist()
    pred_labels = [id2label[p] for p in predictions]
    word_ids = encoding.word_ids()

    entities = defaultdict(list)
    current_entity = None
    current_tokens = []
    previous_word_id = None

    for idx, word_id in enumerate(word_ids):
        if word_id is None or word_id == previous_word_id:
            continue

        token = tokens[word_id]
        label = pred_labels[idx]

        if label.startswith("B-"):
            if current_entity:
                entities[current_entity].append(" ".join(current_tokens))
            current_entity = label[2:]
            current_tokens = [token]

        elif label.startswith("I-") and current_entity == label[2:]:
            current_tokens.append(token)

        else:
            if current_entity:
                entities[current_entity].append(" ".join(current_tokens))
                current_entity = None
                current_tokens = []

        previous_word_id = word_id

    if current_entity:
        entities[current_entity].append(" ".join(current_tokens))

    return entities


In [ ]:
def safe_join(entity_list):
    if not entity_list:
        return None
    return "; ".join(list(set(entity_list)))  # duplicate temizle


In [ ]:
df_unique["email"] = None
df_unique["phone"] = None
df_unique["date"] = None


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
def extract_entities_from_text(text, tokenizer, model, id2label, max_length=256):
    tokens = text.split()

    encoding = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        max_length=max_length
    )

    encoding = encoding.to(device)

    with torch.no_grad():
        outputs = model(**encoding)

    predictions = torch.argmax(outputs.logits, dim=-1)[0].tolist()
    pred_labels = [id2label[p] for p in predictions]

    word_ids = encoding.word_ids()

    from collections import defaultdict
    entities = defaultdict(list)

    current_entity = None
    current_tokens = []
    previous_word_id = None

    for idx, word_id in enumerate(word_ids):
        if word_id is None or word_id == previous_word_id:
            continue

        token = tokens[word_id]
        label = pred_labels[idx]

        if label.startswith("B-"):
            if current_entity:
                entities[current_entity].append(" ".join(current_tokens))
            current_entity = label[2:]
            current_tokens = [token]

        elif label.startswith("I-") and current_entity == label[2:]:
            current_tokens.append(token)

        else:
            if current_entity:
                entities[current_entity].append(" ".join(current_tokens))
                current_entity = None
                current_tokens = []

        previous_word_id = word_id

    if current_entity:
        entities[current_entity].append(" ".join(current_tokens))

    return entities


In [ ]:
for idx, row in df_unique.iterrows():
    text = row["Text"]

    ents = extract_entities_from_text(
        text=text,
        tokenizer=tokenizer,
        model=model,
        id2label=id2label
    )

    df_unique.at[idx, "email"] = safe_join(ents.get("EMAIL"))
    df_unique.at[idx, "phone"] = safe_join(ents.get("PHONE"))
    df_unique.at[idx, "date"]  = safe_join(ents.get("DATE"))

    if idx % 500 == 0:
        print(f"{idx} CV processed")


0 CV processed
500 CV processed
1000 CV processed
1500 CV processed
2000 CV processed
2500 CV processed
3000 CV processed
3500 CV processed
4000 CV processed
4500 CV processed
5500 CV processed
6000 CV processed
6500 CV processed
7000 CV processed
7500 CV processed
8000 CV processed
8500 CV processed
9000 CV processed
9500 CV processed
10000 CV processed
10500 CV processed
11000 CV processed
11500 CV processed
12000 CV processed
12500 CV processed
13000 CV processed


In [ ]:
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12175 entries, 0 to 13388
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  12175 non-null  object
 1   Text      12175 non-null  object
 2   email     10615 non-null  object
 3   phone     8516 non-null   object
 4   date      11924 non-null  object
dtypes: object(5)
memory usage: 828.7+ KB


In [ ]:
df_unique.head(50)

,Category,Text,email,phone,date
0,Accountant,"﻿________________\r\n\r\nEDUCATION\r\nOMBA - Executive Leadership University of Texas\r\n2016-2018\r\nO Bachelor of Science in Accounting Richland College\r\n2005-2008\r\nTRAINING & CERTIFICATIONS\r\nCertified Management Accountant (CMA)\r\nCertified Financial Modeling and Valuation Analyst\r\nCompliance and Anti-Money Laundering (09/2016) American Institute of Banking\r\nCertified Public Account (CPA)\r\nLean Six Sigma Green Belt Certified\r\nTrade Products Financial Regulations (08/2016) American Institute of Banking\r\nACHIEVEMENTS\r\nSpeaker | Bringing out the Leader within YOU (08/2019)\r\nSuccessfully presented an empowering speech on leadership to a 500+ participants.\r\nSpeaker | Dallas Convention of CPAs (03/2019) Successfully delivered a seminar to 3K+ CPAs and convention guests\r\nTEACHING EXPERIENCE\r\nOnline Teacher - Udemy (2017)\r\nTaught Online Accounting for Non-Accountant Course through Udemy and similar online teaching platforms.\r\n- Developed effective teaching modules, materials, and curriculum for the target students.\r\nTook feedbacks from students to assist in improving the teaching methodology and materials.\r\nPROFESSIONAL MEMBERSHIPS & AFFILIATIONS\r\nAmerican Society of Executives (2018 - Present)\r\nTECHNICAL SKILLS\r\nQuickbooks\r\nERP (SAP & Oracle)\r\nHyperion\r\nLANGUAGES\r\nEnglish\r\nNative\r\nFellow Chartered Accountant (2011 - Present)\r\nMS Office\r\nSQL\r\nIBM Cognos\r\nGerman\r\nSpanish\r\nFrench\r\nFull Professional Proficiency\r\nLimited Working Proficiency\r\nLimited Working Proficiency\r\nINTERESTS\r\nArtificial Intelligence\r\nChess\r\nNnovoresume.com\r\nSailing\r\nCryptocurrencies\r\nPage 2 of 2",None,None,(2011 - Present); 2016-2018; (2018 -; (09/2016); (08/2016); (03/2019); 2005-2008; (2017); (08/2019)
1,Accountant,"﻿________________\r\n\r\nHOWARD GERRARD\r\nAccountant\r\nDeyjob.com, Birmingham\r\n0123 456 7890 info@dayjob.com\r\nin Linkedin/name\r\nWORK EXPERIENCE\r\nCompany name\r\nACCOUNTANT - Location\r\nSUMMARY\r\nHoward is considered an expert at managing a company's finances and looking after its accounts. He is able to perform mathematical calculations with speed as well as accuracy and can be relied upon to provide accurate and trustworthy information about financial transactions. He is confident in his ability to play a strategic role in any business.\r\nEDUCATION\r\nUniversity\r\nCourse details\r\n2010-2013\r\n2016-Present\r\nResponsible for ensuring the smooth running of the accounts department. Advising and updating the Director on irregularities or concerns about invoices & financial matters.\r\n• Keeping financial control records for expenditures, allotments and receipts etc.\r\n• Verifying financial documents for validity, accuracy and completeness.\r\n• Managing the receipts and deposits of cash and bank transactions.\r\n• Ensuring that all invoices and staff reimbursements are paid accurately.\r\nCompany name\r\nACCOUNTANT - Location\r\n2015-2016\r\nMaintaining the quality of financial controls. Organising and co-ordinating the work of junior accounting personnel. Locating errors in accounting records.\r\nAssisting colleagues in other departments with preparation of the annual budget.\r\n• Implementing departmental accounting policies, procedures and processes. Using financial tools and techniques to evaluate business opportunities.\r\n• Presenting accounting data in a clear and understandable manner.\r\nCompany name\r\nACCOUNTANT-Location\r\n2014-2015\r\nReviewing monthly departmental Profit and Losses. Assisting with financial and tax audits. Detecting and preventing fraud. Controlling income and expenditure.\r\n■Documenting internal financial controls and producing detailed expenditure reports. ■Completing VAT returns as well as writing cheques and setting up BACS payments. • Writing large cheques for suppliers and setting up BACS payments for clients. • Giving professional accounting advice to those who ask for 

--------------------**EXPERİENCE ÇIKARMA**-------------------------

In [ ]:
MIN_REASONABLE_YEAR = 1950
MAX_TOTAL_MONTHS = 900
CONTEXT_WINDOW = 120

In [ ]:
JOB_KEYWORDS = [
    'company','inc','ltd','llc','co.','corp','manager','director','engineer','accountant',
    'developer','analyst','consultant','assistant','intern','supervisor','partner','associate',
    'president','ceo','cto','cfo','lead','senior','junior','head','works at',' at '
]
EXP_HEADERS = [r'\bwork experience\b', r'\bprofessional experience\b', r'\bexperience\b',
               r'\bemployment history\b', r'\bwork history\b', r'\bcareer history\b',
               r'\bprofessional background\b', r'\bemployment\b']
STOP_HEADERS = [r'\beducation\b', r'\bskills\b', r'\bcertifications\b', r'\bcourses\b',
                r'\bprojects\b', r'\bsummary\b', r'\bcontact\b', r'\bprofile\b',
                r'\bpublications\b', r'\bhonors\b', r'\blicenses\b', r'\binterests\b',
                r'\bawards\b', r'\bobjective\b', r'\breferences\b']

In [ ]:
MONTHS = {
 'jan':1,'january':1,'feb':2,'february':2,'mar':3,'march':3,'apr':4,'april':4,
 'may':5,'jun':6,'june':6,'jul':7,'july':7,'aug':8,'august':8,'sep':9,'sept':9,'september':9,
 'oct':10,'october':10,'nov':11,'november':11,'dec':12,'december':12
}

In [ ]:
def normalize_text(t):
    if not isinstance(t, str): return ""
    t = t.replace('\r','\n')
    t = re.sub(r'[–—−]', '-', t)
    t = re.sub(r'\t', ' ', t)
    return t

In [ ]:
def two_to_four_digit_year(y):
    y = int(y)
    if y >= 100: return y
    cur = datetime.now().year % 100
    if y <= cur:
        return 2000 + y
    else:
        return 1900 + y

In [ ]:
def clamp_year(y):
    try:
        y = int(y)
    except:
        return None
    current = datetime.now().year
    if 1900 <= y <= current + 1:
        return y
    return None

In [ ]:
def month_str_to_int(s):
    if not s: return None
    s = s.strip().lower()
    if s.isdigit():
        v = int(s)
        if 1 <= v <= 12: return v
        return None
    return MONTHS.get(s[:9])

In [ ]:
def parse_month_year_pair(month_tok, year_tok, default='start'):
    if not year_tok:
        return None
    if len(year_tok) == 2:
        y = two_to_four_digit_year(year_tok)
    else:
        try:
            y = int(year_tok)
        except:
            return None
    y = clamp_year(y)
    if y is None:
        return None
    if month_tok:
        m = month_str_to_int(month_tok)
        if not m:
            return datetime(y, 1 if default=='start' else 12, 1)
        return datetime(y, m, 1)
    else:
        return datetime(y, 1 if default=='start' else 12, 1)


In [ ]:
# We'll capture multiple common date-range formats and return (start_dt,end_dt, span)
_months_regex = r'(' + r'|'.join(MONTHS.keys()) + r'|\d{1,2})\.?'
# patterns:
P1 = re.compile(r'\b' + _months_regex + r'\s+(\d{2,4})\s*(?:-|to|until)\s*(' + _months_regex + r'\s+(\d{2,4})|(present|current|ongoing)|(\d{4}))', re.IGNORECASE)
P2 = re.compile(r'\b(0?[1-9]|1[0-2])[./-](\d{2,4})\s*(?:-|to|until)?\s*(present|current|ongoing|(0?[1-9]|1[0-2])[./-](\d{2,4})|\d{4})', re.IGNORECASE)
P3 = re.compile(r'\b(\d{4})\s*(?:-|to|until)\s*(present|current|ongoing|\d{4})\b', re.IGNORECASE)
P4 = re.compile(r'\b(' + r'|'.join(MONTHS.keys()) + r'|\d{1,2})\.?\s+(\d{4})\s+(' + r'|'.join(MONTHS.keys()) + r'|\d{1,2})\.?\s+(\d{4})\b', re.IGNORECASE)
# also single month-year or year
SINGLE_MY = re.compile(r'\b(' + r'|'.join(MONTHS.keys()) + r'|\d{1,2})\.?\s+(\d{4})\b', re.IGNORECASE)
SINGLE_Y  = re.compile(r'\b(19|20)\d{2}\b')

In [ ]:
def extract_date_ranges_with_spans(text):
    t = normalize_text(text)
    low = t.lower()
    now = datetime.now()
    ranges = []
    # p1
    for m in P1.finditer(low):
        span = m.span()
        s_m, s_y = m.group(1), m.group(2)
        if m.group(4):  # present
            start = parse_month_year_pair(s_m, s_y, 'start')
            end = now
        elif m.group(5):  # year-only end (group indices shifted vs earlier)
            start = parse_month_year_pair(s_m, s_y, 'start')
            end = parse_month_year_pair(None, m.group(5), 'end')
        else:
            # groups: 3 and 4 maybe: (e_m, e_y)
            # fallback to search subgroups
            try:
                e_m = m.group(3)
                e_y = m.group(4)
                start = parse_month_year_pair(s_m, s_y, 'start')
                end = parse_month_year_pair(e_m, e_y, 'end')
            except:
                continue
        if start and end and start <= end:
            if start.year >= MIN_REASONABLE_YEAR or (end >= datetime(datetime.now().year,1,1)):
                ranges.append({"start_dt": start.replace(day=1), "end_dt": end.replace(day=1), "span": span})
    # p2
    for m in P2.finditer(low):
        span = m.span()
        sm, sy = m.group(1), m.group(2)
        try:
            if m.group(3) and m.group(3) in ('present','current','ongoing'):
                start = parse_month_year_pair(sm, sy, 'start')
                end = now
            elif m.group(4) and m.group(5):
                em, ey = m.group(4), m.group(5)
                start = parse_month_year_pair(sm, sy, 'start')
                end = parse_month_year_pair(em, ey, 'end')
            elif m.group(3) and re.fullmatch(r'\d{4}', m.group(3)):
                start = parse_month_year_pair(sm, sy, 'start')
                end = parse_month_year_pair(None, m.group(3), 'end')
            else:
                continue
            if start and end and start <= end:
                if start.year >= MIN_REASONABLE_YEAR or end >= datetime(datetime.now().year,1,1):
                    ranges.append({"start_dt": start.replace(day=1), "end_dt": end.replace(day=1), "span": span})
        except Exception:
            continue
    # p3
    for m in P3.finditer(low):
        span = m.span()
        sy = m.group(1); ey = m.group(2)
        start = parse_month_year_pair(None, sy, 'start')
        if ey.lower() in ('present','current','ongoing'):
            end = now
        else:
            end = parse_month_year_pair(None, ey, 'end')
        if start and end and start <= end:
            if start.year >= MIN_REASONABLE_YEAR or 'present' in ey.lower():
                ranges.append({"start_dt": start.replace(day=1), "end_dt": end.replace(day=1), "span": span})
    # p4 adjacent month-year pairs without dash
    for m in P4.finditer(low):
        span = m.span()
        start = parse_month_year_pair(m.group(1), m.group(2), 'start')
        end = parse_month_year_pair(m.group(3), m.group(4), 'end')
        if start and end and start <= end:
            if start.year >= MIN_REASONABLE_YEAR:
                ranges.append({"start_dt": start.replace(day=1), "end_dt": end.replace(day=1), "span": span})
    # fallback: single month-year or year (as single-date entries)
    # (we will not use them as ranges directly, but can be helpful)
    for m in SINGLE_MY.finditer(low):
        span = m.span()
        start = parse_month_year_pair(m.group(1), m.group(2), 'start')
        if start and start.year >= MIN_REASONABLE_YEAR:
            ranges.append({"start_dt": start.replace(day=1), "end_dt": start.replace(day=1), "span": span})
    for m in SINGLE_Y.finditer(low):
        span = m.span()
        start = parse_month_year_pair(None, m.group(0), 'start')
        if start and start.year >= MIN_REASONABLE_YEAR:
            ranges.append({"start_dt": start.replace(day=1), "end_dt": start.replace(day=1), "span": span})
    # return normalized unique by span start
    # remove duplicates by span
    seen_spans = set()
    final = []
    for r in ranges:
        if r["span"] in seen_spans: continue
        seen_spans.add(r["span"])
        final.append(r)
    return final

In [ ]:
# months between inclusive (same as seninki)
def months_between_inclusive(start_dt, end_dt):
    return (end_dt.year - start_dt.year) * 12 + (end_dt.month - start_dt.month) + 1

In [ ]:
# merge overlapping ranges (list of dicts with start_dt,end_dt)
def merge_ranges_list(ranges):
    if not ranges:
        return []
    ranges_sorted = sorted(ranges, key=lambda x: x["start_dt"])
    merged = []
    cur = ranges_sorted[0].copy()
    for r in ranges_sorted[1:]:
        if r["start_dt"] <= cur["end_dt"]:
            if r["end_dt"] > cur["end_dt"]:
                cur["end_dt"] = r["end_dt"]
        else:
            merged.append(cur)
            cur = r.copy()
    merged.append(cur)
    return merged

In [ ]:
# company extraction from context (heuristic)
COMPANY_PATTERNS = [
    re.compile(r'\bat\s+([A-Z][A-Za-z&\.\- ]{2,70})'),  # at Acme Corp
    re.compile(r'([A-Z][A-Za-z&\.\- ]{2,70})\s+(?:Inc|Ltd|LLC|Corp|Company|PLC|GmbH)\b'),
    re.compile(r'([A-Z][A-Za-z ]{2,70}University)\b'),
]

def extract_company_from_context(context):
    if not context:
        return None

    # önce satır satır dene (en önemli kısım)
    for line in context.split('\n'):
        line = line.strip()
        if len(line) < 3:
            continue

        for p in COMPANY_PATTERNS:
            m = p.search(line)
            if m:
                c = m.group(1).strip()
                c = re.sub(r'\s{2,}', ' ', c)
                return c

    # satır bazlı bulunmazsa tüm context'te dene (fallback)
    for p in COMPANY_PATTERNS:
        m = p.search(context)
        if m:
            c = m.group(1).strip()
            c = re.sub(r'\s{2,}', ' ', c)
            return c

    return None


In [ ]:
# title extraction: look for job keywords in the context and return short phrase
def extract_date_ranges_relaxed(text):
    """
    High-recall date extraction:
    - uses extract_date_ranges_with_spans
    - does NOT apply job-context filtering
    """
    ranges = extract_date_ranges_with_spans(text)

    final = []
    for r in ranges:
        duration = months_between_inclusive(r["start_dt"], r["end_dt"])
        if duration >= 2:   # en az 2 ay
            final.append(r)

    return final

In [ ]:
TITLE_BLACKLIST = [
    'chartered', 'certified', 'certificate', 'certification',
    'bachelor', 'master', 'phd', 'degree'
]
def extract_title_from_context(context):
    ctx = context.lower()
    for b in TITLE_BLACKLIST:
        if b in ctx:
            return None
    # first: try to find phrase containing a job keyword
    for kw in JOB_KEYWORDS:
        if kw in ctx:
            # return the sentence that contains it (split on line breaks or punctuation)
            segs = re.split(r'[\n\.\,;-]', context)
            for s in segs:
                if kw in s.lower():
                    cand = s.strip()
                    # trim to reasonable length
                    if 3 <= len(cand) <= 120:
                        return re.sub(r'\s{2,}', ' ', cand)
    # fallback: look for capitalized short phrases with common title words
    m = re.search(r'([A-Z][A-Za-z/]{2,40}\s+(?:Engineer|Developer|Analyst|Manager|Consultant|Director|Lead|Specialist|Intern))', context)
    if m:
        return m.group(1).strip()
    return None

In [ ]:
def find_section_span(text, headers):
    """
    text: normalized text (string)
    headers: list of regex header pattern'ları (küçük harfli arama için uygun)
    returns: (start_idx, start_end_idx) of earliest found header or (None, None)
    """
    low = text.lower()
    first = None
    first_end = None
    for h in headers:
        m = re.search(h, low)
        if m:
            if first is None or m.start() < first:
                first = m.start()
                first_end = m.end()
    return (first, first_end)

def extract_experience_section(text):
    """
    CV metninden 'experience' bölümünü çıkarır.
    - Eğer explicit bir başlık bulunursa, o başlıktan sonraki metni STOP_HEADERS'e kadar alır.
    - Çıktı: eğer bölüm birden fazla paragraftan oluşuyorsa liste, değilse string döner.
    - Eğer deneyim başlığı yoksa boş string döner.
    """
    txt = normalize_text(text)
    start, start_end = find_section_span(txt, EXP_HEADERS)
    if start is None:
        return ""  # explicit experience header yok

    # stop header araması (start_end sonrasından)
    low = txt.lower()
    stop_pos = None
    for s in STOP_HEADERS:
        m = re.search(s, low[start_end:])
        if m:
            pos = start_end + m.start()
            if stop_pos is None or pos < stop_pos:
                stop_pos = pos

    body = txt[start_end:stop_pos] if stop_pos else txt[start_end:]

    # paragraf bazlı ayır (iki veya daha fazla boş satır ile)
    parts = [p.strip() for p in re.split(r'\n{2,}', body) if p.strip()]

    if not parts:
        return ""
    if len(parts) == 1:
        return parts[0]
    return parts


In [ ]:
# main: build entries from text using date ranges and context heuristics

def build_experience_entries(text):
    txt = normalize_text(text)

    exp_section = extract_experience_section(txt)

    if isinstance(exp_section, list):
        exp_section_text = "\n".join(exp_section)
    else:
        exp_section_text = exp_section

    base_text = exp_section_text if exp_section_text.strip() else txt

    date_ranges = extract_date_ranges_relaxed(base_text)

    entries = []
    for dr in date_ranges:
        s_dt, e_dt = dr["start_dt"], dr["end_dt"]
        span = dr["span"]

        s_char = max(0, span[0] - CONTEXT_WINDOW)
        e_char = min(len(base_text), span[1] + CONTEXT_WINDOW)
        context = base_text[s_char:e_char]

        title = extract_title_from_context(context)
        company = extract_company_from_context(context)

        entries.append({
            "start_dt": s_dt,
            "end_dt": e_dt,
            "title": title,
            "company": company,
            "context": context.strip()
        })

    if not entries:
        return []

    merged_ranges = merge_ranges_list(
        [{"start_dt": e["start_dt"], "end_dt": e["end_dt"]} for e in entries]
    )

    final = []
    for m in merged_ranges:
        metas = [e for e in entries if not (e["end_dt"] < m["start_dt"] or e["start_dt"] > m["end_dt"])]

        final.append({
            "start_dt": m["start_dt"],
            "end_dt": m["end_dt"],
            "title": next((x["title"] for x in metas if x["title"]), None),
            "company": next((x["company"] for x in metas if x["company"]), None),
            "contexts": [x["context"] for x in metas]
        })

    return final


In [ ]:
def format_entry(e):
    sd = e["start_dt"].strftime("%Y-%m") if e["start_dt"] else None
    ed = e["end_dt"].strftime("%Y-%m") if e["end_dt"] else None
    return {"title": e.get("title"), "company": e.get("company"), "start": sd, "end": ed}

# Kullanım örneği (ilk 5 CV üzerinde hızlı test)
for i in range(5):
    ents = build_experience_entries(df_unique.iloc[i]["Text"])
    print(f"CV {i}: {len(ents)} entries")
    for e in ents:
        print(
            e["start_dt"].strftime("%Y-%m"),
            "→",
            e["end_dt"].strftime("%Y-%m"),
            "|",
            e["title"],
            "|",
            e["company"]
        )
    print("------")

In [ ]:
def merge_date_entries(entries):
    entries = sorted(entries, key=lambda x: x["start_dt"])
    merged = []

    cur = entries[0]
    for nxt in entries[1:]:
        if nxt["start_dt"] <= cur["end_dt"]:
            cur["end_dt"] = max(cur["end_dt"], nxt["end_dt"])
            cur["title"] = cur["title"] or nxt["title"]
            cur["company"] = cur["company"] or nxt["company"]
        else:
            merged.append(cur)
            cur = nxt

    merged.append(cur)
    return merged

In [ ]:
def total_months_from_entries(entries):
    total = 0
    for e in entries:
        total += months_between_inclusive(e["start_dt"], e["end_dt"])
    return min(total, MAX_TOTAL_MONTHS)

In [ ]:
# tek bir CV test
cv_text = df_unique.iloc[0]["Text"]
entries = build_experience_entries(cv_text)
print(entries)

# tüm df_unique'e uygulama (yavaş olabilir; örnek olarak ilk 200 yap)
df_unique['experience_entries'] = df_unique['Text'].apply(build_experience_entries)
df_unique['experience_total_months_parsed'] = df_unique['experience_entries'].apply(total_months_from_entries)

# hızlı kontrol
df_unique[['experience_entries','experience_total_months_parsed']].head()


[{'start_dt': datetime.datetime(2018, 1, 1, 0, 0), 'end_dt': datetime.datetime(2026, 1, 1, 21, 26, 25, 120672), 'title': None, 'company': None, 'contexts': ['proving the teaching methodology and materials.\nPROFESSIONAL MEMBERSHIPS & AFFILIATIONS\nAmerican Society of Executives (2018 - Present)\nTECHNICAL']}]


,experience_entries,experience_total_months_parsed
0,"[{'start_dt': 2018-01-01 00:00:00, 'end_dt': 2026-01-01 21:26:25.122165, 'title': None, 'company': None, 'contexts': ['proving the teaching methodology and materials. PROFESSIONAL MEMBERSHIPS & AFFILIATIONS American Society of Executives (2018 - Present) TECHNICAL']}]",97
1,[],0
2,"[{'start_dt': 2018-09-01 00:00:00, 'end_dt': 2019-12-01 00:00:00, 'title': None, 'company': None, 'contexts': ['approvals for periodic reconciliation of the account. Bank Accountanat Mediterraneo GROUP For Importing And Exporting. Sep 2018 Dec 2019 • Prepared journal entries, complete general ledger operations, monthly, year-end closings and draw up financial report']}, {'start_dt': 2020-01-01 00:00:00, 'end_dt': 2026-01-01 21:26:25.124548, 'title': '• See the contract between the company and the customer and review the terms of payment and guarantees and any other co', 'company': None, 'contexts': ['Senior Accountant & Customer Reference Mediterraneo Group For Importing and Exporting Jan 2020-Present • See the contract between the company and the customer and review the terms of payment and guarantees and any other co']}]",89
3,"[{'start_dt': 2011-01-01 00:00:00, 'end_dt': 2026-01-01 21:26:25.127147, 'title': 'Accountant III', 'company': None, 'contexts': ['nts of up to 30% in terms of processing speed and accuracy. Accountant III, BZG Bank Full Los Angeles Jan 2013 Mar 2017 Jan 2011 - Dec 2012 Implemented a cash conversion model for six branches and ten ATMs that saved $40,000 a year in processing expenses. Ove', 'May 2017 Apr 2019 - Experienced Certified Public Accountant with 7+ years of experience in financial reporting, reconciliations and P&L s', 'lting in improvements of up to 30% in terms of processing speed and accuracy. Accountant III, BZG Bank Full Los Angeles Jan 2013 Mar 2017 Jan 2011 - Dec 2012 Implemented a cash conversion model for six branches and ten ATMs that saved $40,000 a year in proc']}]",181
4,"[{'start_dt': 2016-01-01 00:00:00, 'end_dt': 2026-01-01 21:26:25.129239, 'title': 'Senior Accountant', 'company': None, 'contexts': ['an early stage financial consumer startup. WORK EXPERIENCE Teachers Pay Teachers Senior Accountant New York, NY | July 2016 - current • Established best practices for revenue recognition in compliance with ASC 606 through 50% YoY revenue growth • Built']}]",121


In [ ]:
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12175 entries, 0 to 13388
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Category                        12175 non-null  object
 1   Text                            12175 non-null  object
 2   email                           10615 non-null  object
 3   phone                           8516 non-null   object
 4   date                            11924 non-null  object
 5   experience_entries              12175 non-null  object
 6   experience_total_months_parsed  12175 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 1019.0+ KB


In [ ]:
df_unique.head(29)

,Category,Text,email,phone,date,experience_entries,experience_total_months_parsed
0,Accountant,"﻿________________\r\n\r\nEDUCATION\r\nOMBA - Executive Leadership University of Texas\r\n2016-2018\r\nO Bachelor of Science in Accounting Richland College\r\n2005-2008\r\nTRAINING & CERTIFICATIONS\r\nCertified Management Accountant (CMA)\r\nCertified Financial Modeling and Valuation Analyst\r\nCompliance and Anti-Money Laundering (09/2016) American Institute of Banking\r\nCertified Public Account (CPA)\r\nLean Six Sigma Green Belt Certified\r\nTrade Products Financial Regulations (08/2016) American Institute of Banking\r\nACHIEVEMENTS\r\nSpeaker | Bringing out the Leader within YOU (08/2019)\r\nSuccessfully presented an empowering speech on leadership to a 500+ participants.\r\nSpeaker | Dallas Convention of CPAs (03/2019) Successfully delivered a seminar to 3K+ CPAs and convention guests\r\nTEACHING EXPERIENCE\r\nOnline Teacher - Udemy (2017)\r\nTaught Online Accounting for Non-Accountant Course through Udemy and similar online teaching platforms.\r\n- Developed effective teaching modules, materials, and curriculum for the target students.\r\nTook feedbacks from students to assist in improving the teaching methodology and materials.\r\nPROFESSIONAL MEMBERSHIPS & AFFILIATIONS\r\nAmerican Society of Executives (2018 - Present)\r\nTECHNICAL SKILLS\r\nQuickbooks\r\nERP (SAP & Oracle)\r\nHyperion\r\nLANGUAGES\r\nEnglish\r\nNative\r\nFellow Chartered Accountant (2011 - Present)\r\nMS Office\r\nSQL\r\nIBM Cognos\r\nGerman\r\nSpanish\r\nFrench\r\nFull Professional Proficiency\r\nLimited Working Proficiency\r\nLimited Working Proficiency\r\nINTERESTS\r\nArtificial Intelligence\r\nChess\r\nNnovoresume.com\r\nSailing\r\nCryptocurrencies\r\nPage 2 of 2",None,None,(2011 - Present); 2016-2018; (2018 -; (09/2016); (08/2016); (03/2019); 2005-2008; (2017); (08/2019),"[{'start_dt': 2018-01-01 00:00:00, 'end_dt': 2026-01-01 21:26:25.122165, 'title': None, 'company': None, 'contexts': ['proving the teaching methodology and materials. PROFESSIONAL MEMBERSHIPS & AFFILIATIONS American Society of Executives (2018 - Present) TECHNICAL']}]",97
1,Accountant,"﻿________________\r\n\r\nHOWARD GERRARD\r\nAccountant\r\nDeyjob.com, Birmingham\r\n0123 456 7890 info@dayjob.com\r\nin Linkedin/name\r\nWORK EXPERIENCE\r\nCompany name\r\nACCOUNTANT - Location\r\nSUMMARY\r\nHoward is considered an expert at managing a company's finances and looking after its accounts. He is able to perform mathematical calculations with speed as well as accuracy and can be relied upon to provide accurate and trustworthy information about financial transactions. He is confident in his ability to play a strategic role in any business.\r\nEDUCATION\r\nUniversity\r\nCourse details\r\n2010-2013\r\n2016-Present\r\nResponsible for ensuring the smooth running of the accounts department. Advising and updating the Director on irregularities or concerns about invoices & financial matters.\r\n• Keeping financial control records for expenditures, allotments and receipts etc.\r\n• Verifying financial documents for validity, accuracy and completeness.\r\n• Managing the receipts and deposits of cash and bank transactions.\r\n• Ensuring that all invoices and staff reimbursements are paid accurately.\r\nCompany name\r\nACCOUNTANT - Location\r\n2015-2016\r\nMaintaining the quality of financial controls. Organising and co-ordinating the work of junior accounting personnel. Locating errors in accounting records.\r\nAssisting colleagues in other departments with preparation of the annual budget.\r\n• Implementing departmental accounting policies, procedures and processes. Using financial tools and techniques to evaluate business opportunities.\r\n• Presenting accounting data in a clear and understandable manner.\r\nCompany name\r\nACCOUNTANT-Location\r\n2014-2015\r\nReviewing monthly departmental Profit and Losses. Assisting with financial and tax audits. Detecting and preventing fraud. Controlling income and

-----------------------**EDUCATİON ÇIKARMA-**-----------------------------

In [ ]:
CATEGORY_DEFAULT_EDU = {

    # ---- TECH / SOFTWARE ----
    "Data Science": [
        "Bachelor in Computer Science",
        "Bachelor in Data Science",
        "Bachelor in Statistics",
        "Bachelor in Mathematics"
    ],

    "Python Developer": [
        "Bachelor in Computer Science",
        "Bachelor in Software Engineering",
        "Bachelor in Information Technology"
    ],

    "Java Developer": [
        "Bachelor in Computer Science",
        "Bachelor in Software Engineering",
        "Bachelor in Information Technology"
    ],

    "DotNet Developer": [
        "Bachelor in Computer Science",
        "Bachelor in Software Engineering",
        "Bachelor in Information Technology"
    ],

    "React Developer": [
        "Bachelor in Computer Science",
        "Bachelor in Software Engineering",
        "Bachelor in Information Technology"
    ],

    "Web Designing": [
        "Bachelor in Computer Science",
        "Bachelor in Web Design",
        "Bachelor in Graphic Design",
        "Bachelor in Information Technology"
    ],

    "DevOps": [
        "Bachelor in Computer Engineering",
        "Bachelor in Software Engineering",
        "Bachelor in Information Technology"
    ],

    "Database": [
        "Bachelor in Computer Science",
        "Bachelor in Information Systems",
        "Bachelor in Information Technology"
    ],

    "SQL Developer": [
        "Bachelor in Computer Science",
        "Bachelor in Information Technology"
    ],

    "ETL Developer": [
        "Bachelor in Computer Science",
        "Bachelor in Information Systems",
        "Bachelor in Data Engineering"
    ],

    "Blockchain": [
        "Bachelor in Computer Science",
        "Bachelor in Software Engineering",
        "Bachelor in Information Technology"
    ],

    "Information Technology": [
        "Bachelor in Information Technology",
        "Bachelor in Computer Science"
    ],

    # ---- ENGINEERING ----
    "Mechanical Engineer": [
        "Bachelor in Mechanical Engineering"
    ],

    "Electrical Engineering": [
        "Bachelor in Electrical Engineering"
    ],

    "Civil Engineer": [
        "Bachelor in Civil Engineering"
    ],

    "Network Security Engineer": [
        "Bachelor in Computer Engineering",
        "Bachelor in Cyber Security",
        "Bachelor in Information Technology"
    ],

    "Architecture": [
        "Bachelor in Architecture"
    ],

    # ---- BUSINESS / MANAGEMENT ----
    "Business Analyst": [
        "Bachelor in Business Administration",
        "Bachelor in Management Information Systems",
        "Bachelor in Industrial Engineering"
    ],

    "Management": [
        "Bachelor in Business Administration",
        "Bachelor in Management"
    ],

    "Operations Manager": [
        "Bachelor in Business Administration",
        "Bachelor in Industrial Engineering"
    ],

    "PMO": [
        "Bachelor in Business Administration",
        "Bachelor in Industrial Engineering",
        "Bachelor in Project Management"
    ],

    "Consultant": [
        "Bachelor in Business Administration",
        "Bachelor in Economics",
        "Bachelor in Engineering"
    ],

    # ---- FINANCE / SALES ----
    "Finance": [
        "Bachelor in Finance",
        "Bachelor in Accounting",
        "Bachelor in Economics",
        "Bachelor in Business Administration"
    ],

    "Accountant": [
        "Bachelor in Accounting",
        "Bachelor in Finance",
        "Bachelor in Commerce"
    ],

    "Banking": [
        "Bachelor in Finance",
        "Bachelor in Banking",
        "Bachelor in Economics",
        "Bachelor in Commerce"
    ],

    "Sales": [
        "Bachelor in Business Administration",
        "Bachelor in Marketing",
        "Bachelor in Commerce"
    ],

    # ---- HR / LEGAL ----
    "Human Resources": [
        "Bachelor in Human Resources",
        "Bachelor in Business Administration",
        "Bachelor in Psychology"
    ],

    "Advocate": [
        "Bachelor in Law (LLB)"
    ],

    # ---- CREATIVE ----
    "Arts": [
        "Bachelor in Fine Arts",
        "Bachelor in Visual Arts",
        "Bachelor in Arts"
    ],

    "Designing": [
        "Bachelor in Graphic Design",
        "Bachelor in Visual Communication",
        "Bachelor in Fine Arts"
    ],

    "Digital Media": [
        "Bachelor in Digital Media",
        "Bachelor in Communication",
        "Bachelor in Media Studies"
    ],

    "Public Relations": [
        "Bachelor in Public Relations",
        "Bachelor in Communication",
        "Bachelor in Journalism"
    ],

    # ---- INDUSTRY / OTHER ----
    "Health and Fitness": [
        "Bachelor in Sports Science",
        "Bachelor in Physical Education",
        "Bachelor in Health Sciences"
    ],

    "Aviation": [
        "Bachelor in Aviation",
        "Bachelor in Aeronautical Engineering",
        "Bachelor in Engineering"
    ],

    "Automobile": [
        "Bachelor in Automobile Engineering",
        "Bachelor in Mechanical Engineering"
    ],

    "Building and Construction": [
        "Bachelor in Civil Engineering",
        "Bachelor in Construction Management"
    ],

    "Agriculture": [
        "Bachelor in Agriculture",
        "Bachelor in Agricultural Engineering"
    ],

    "Food and Beverages": [
        "Bachelor in Hospitality Management",
        "Bachelor in Food Technology"
    ],

    "Education": [
        "Bachelor in Education",
        "Bachelor in Arts",
        "Bachelor in Science"
    ],

    "BPO": [
        "Bachelor in Arts",
        "Bachelor in Commerce",
        "Bachelor in Business Administration"
    ]
}


In [ ]:
import unicodedata
def normalize_text(text):
    if not isinstance(text, str):
        return ""
    text = unicodedata.normalize("NFKC", text)
    text = text.replace("\r", "\n")
    text = re.sub(r"\n\s*\n+", "\n\n", text)
    text = "\n".join([line.strip() for line in text.split("\n")])
    return text

In [ ]:
EDU_SECTION_PATTERN = re.compile(
    r'(?P<header>(?:^|\n)\s*'
    r'(education details|education and training|education|academic details|qualification|academic background)'
    r'[\s:\-•]*)'
    r'(?P<body>[\s\S]{0,800}?)'
    r'(?=(\n\s*(experience|work experience|skills|projects|certifications|achievements|$)))',
    re.IGNORECASE
)

EDU_LINE_PATTERN = re.compile(
    r"(b\.?e\.?|bachelor|master|m\.?sc|msc|b\.?sc|bsc|mba|mms|llb|llm|diploma|ssc|pgp|hsc)",
    re.IGNORECASE
)

EDU_INSTITUTE_PATTERN = re.compile(
    r"(university|college|institute|polytechnic|school|board)",
    re.IGNORECASE
)

DEGREE_PATTERN = re.compile(
    r"""
    (
        (bachelor|master|ph\.?d|doctorate|diploma)
        [^.\n]{0,80}
        |
        \b(b\.?e|b\.?tech|m\.?tech|mca|mba|m\.?sc|b\.?sc)\b
        [^.\n]{0,80}
    )
    """,
    re.IGNORECASE | re.VERBOSE
)

DEGREE_ALIASES = [
    r"\bbe\b", r"\bb\.e\b", r"\bbachelor\b",
    r"\bbtech\b", r"\bb\.tech\b",
    r"\bmca\b", r"\bbca\b",
    r"\bma\b", r"\bba\b",
    r"\bmtech\b", r"\bm\.tech\b",
    r"\bmsc\b", r"\bbsc\b",
    r"\bllb\b", r"\bphd\b", r"\bdoctorate\b"
]

In [ ]:
def extract_loose_degree_lines(text):
    results = []
    for line in text.splitlines():
        l = line.strip().lower()
        if 10 < len(l) < 200:
            for deg in DEGREE_ALIASES:
                if re.search(deg, l):
                    results.append(line.strip())
                    break
    return results

In [ ]:
def extract_degree_sentences(text):
    return [
        line.strip()
        for line in text.splitlines()
        if 10 < len(line.strip()) < 200 and DEGREE_PATTERN.search(line)
    ]

In [ ]:
def extract_education_lines(text):
    return [
        line.strip()
        for line in text.splitlines()
        if 10 < len(line.strip()) < 200 and EDU_LINE_PATTERN.search(line)
    ]

In [ ]:
def extract_institution_lines(text):
    return [
        line.strip()
        for line in text.splitlines()
        if 10 < len(line.strip()) < 200 and EDU_INSTITUTE_PATTERN.search(line)
    ]

In [ ]:
def extract_education_final(text, category=None):
    text = normalize_text(text)
    education = []

    # SECTION BASED
    for match in EDU_SECTION_PATTERN.finditer(text):
        body = match.group("body")
        parts = [p.strip() for p in re.split(r"\n{2,}", body) if p.strip()]

        for p in parts:
            seg = re.sub(r"\s*\n\s*", " ", p).strip()
            if 10 < len(seg) < 250:
                education.append(seg)

    # LINE BASED FALLBACKS
    education.extend(extract_degree_sentences(text))
    education.extend(extract_loose_degree_lines(text))
    education.extend(extract_institution_lines(text))

    # CLEANUP (ORDER PRESERVED)
    seen = set()
    cleaned = []

    for e in education:
        e_low = e.lower()
        if e_low in seen:
            continue
        if any(k in e_low for k in [
            "work experience", "experience", "skills", "projects",
            "responsibilities", "company", "employment"
        ]):
            continue
        seen.add(e_low)
        cleaned.append(e)

    education = cleaned

    # CATEGORY DEFAULT
    if not education and category in CATEGORY_DEFAULT_EDU:
        education = CATEGORY_DEFAULT_EDU[category]

    # FINAL FALLBACK
    if not education:
        education = ["Not Mentioned"]

    return education

In [ ]:
df_unique['education'] = df_unique['Text'].apply(extract_education_final)

In [ ]:
# Bullet & özel unicode işaretler
BULLET_RE = re.compile(r'[•⚫▪◦●■◆▶►–—]')

# Page, resume footer vs.
PAGE_RE = re.compile(r'\b(page|sayfa)\s*\d+(\s*of\s*\d+)?', re.I)

In [ ]:
def clean_education(edu_list):
    if not isinstance(edu_list, list):
        return ["Not Mentioned"]

    cleaned = []

    for e in edu_list:
        if not isinstance(e, str):
            continue

        t = e.strip()

        # Bullet karakterlerini sil
        t = BULLET_RE.sub(" ", t)

        # Page / sayfa ifadelerini sil
        t = PAGE_RE.sub(" ", t)

        # temel normalize
        t = t.lower()
        t = t.strip(".,;:()[]{}\"'")
        t = re.sub(r"\s{2,}", " ", t).strip()

        # çok kısa veya anlamsız kalanlar
        if not t or len(t) < 3:
            continue

        cleaned.append(t)

    return list(dict.fromkeys(cleaned)) if cleaned else ["Not Mentioned"]



In [ ]:
df_unique['education'] = df_unique['education'].apply(clean_education)

In [ ]:
df_unique[['Category','Text', 'education']].head(150)

,Category,Text,education
0,Accountant,"﻿________________\r\n\r\nEDUCATION\r\nOMBA - Executive Leadership University of Texas\r\n2016-2018\r\nO Bachelor of Science in Accounting Richland College\r\n2005-2008\r\nTRAINING & CERTIFICATIONS\r\nCertified Management Accountant (CMA)\r\nCertified Financial Modeling and Valuation Analyst\r\nCompliance and Anti-Money Laundering (09/2016) American Institute of Banking\r\nCertified Public Account (CPA)\r\nLean Six Sigma Green Belt Certified\r\nTrade Products Financial Regulations (08/2016) American Institute of Banking\r\nACHIEVEMENTS\r\nSpeaker | Bringing out the Leader within YOU (08/2019)\r\nSuccessfully presented an empowering speech on leadership to a 500+ participants.\r\nSpeaker | Dallas Convention of CPAs (03/2019) Successfully delivered a seminar to 3K+ CPAs and convention guests\r\nTEACHING EXPERIENCE\r\nOnline Teacher - Udemy (2017)\r\nTaught Online Accounting for Non-Accountant Course through Udemy and similar online teaching platforms.\r\n- Developed effective teaching modules, materials, and curriculum for the target students.\r\nTook feedbacks from students to assist in improving the teaching methodology and materials.\r\nPROFESSIONAL MEMBERSHIPS & AFFILIATIONS\r\nAmerican Society of Executives (2018 - Present)\r\nTECHNICAL SKILLS\r\nQuickbooks\r\nERP (SAP & Oracle)\r\nHyperion\r\nLANGUAGES\r\nEnglish\r\nNative\r\nFellow Chartered Accountant (2011 - Present)\r\nMS Office\r\nSQL\r\nIBM Cognos\r\nGerman\r\nSpanish\r\nFrench\r\nFull Professional Proficiency\r\nLimited Working Proficiency\r\nLimited Working Proficiency\r\nINTERESTS\r\nArtificial Intelligence\r\nChess\r\nNnovoresume.com\r\nSailing\r\nCryptocurrencies\r\nPage 2 of 2","[omba - executive leadership university of texas, o bachelor of science in accounting richland college, training & certifications, certified management accountant (cma, certified financial modeling and valuation analyst, compliance and anti-money laundering (09/2016) american institute of banking, certified public account (cpa, lean six sigma green belt certified, trade products financial regulations (08/2016) american institute of banking]"
1,Accountant,"﻿________________\r\n\r\nHOWARD GERRARD\r\nAccountant\r\nDeyjob.com, Birmingham\r\n0123 456 7890 info@dayjob.com\r\nin Linkedin/name\r\nWORK EXPERIENCE\r\nCompany name\r\nACCOUNTANT - Location\r\nSUMMARY\r\nHoward is considered an expert at managing a company's finances and looking after its accounts. He is able to perform mathematical calculations with speed as well as accuracy and can be relied upon to provide accurate and trustworthy information about financial transactions. He is confident in his ability to play a strategic role in any business.\r\nEDUCATION\r\nUniversity\r\nCourse details\r\n2010-2013\r\n2016-Present\r\nResponsible for ensuring the smooth running of the accounts department. Advising and updating the Director on irregularities or concerns about invoices & financial matters.\r\n• Keeping financial control records for expenditures, allotments and receipts etc.\r\n• Verifying financial documents for validity, accuracy and completeness.\r\n• Managing the receipts and deposits of cash and bank transactions.\r\n• Ensuring that all invoices and staff reimbursements are paid accurately.\r\nCompany name\r\nACCOUNTANT - Location\r\n2015-2016\r\nMaintaining the quality of financial controls. Organising and co-ordinating the work of junior accounting personnel. Locating errors in accounting records.\r\nAssisting colleagues in other departments with preparation of the annual budget.\r\n• Implementing departmental accounting policies, procedures and processes. Using financial tools and techniques to evaluate business opportunities.\r\n• Presenting accounting data in a clear and understandable manner.\r\nCompany name\r\nACCOUNTANT-Location\r\n2014-2015\r\nReviewing monthly departmental Profit and Losses. Assisting with financial and tax audits. Detecting and preventing fraud. Controlling inco

In [ ]:
(df_unique['education']
 .apply(lambda x: isinstance(x, list) and x == ["not mentioned"])
 .sum())

np.int64(1192)

-----------------------**SKİLL ÇIKARMA**-----------------------------

In [ ]:
SKILL_SECTION_PATTERN = re.compile(
    r"(skills?|technical skills?|core competencies|expertise|technologies|tools)"
    r"[\s:\-•]*([\s\S]{20,800}?)"
    r"(experience|education|projects|certifications|$)",
    re.IGNORECASE
)

In [ ]:
CATEGORY_SKILLS = {

    "Data Science": [
        "python","pandas","numpy","scikit-learn","machine learning",
        "deep learning","nlp","tensorflow","pytorch","sql",
        "statistics","data analysis","feature engineering","modeling"
    ],

    "Python Developer": [
        "python","django","flask","fastapi","sql",
        "rest api","celery","pytest","asyncio"
    ],

    "Java Developer": [
        "java","spring","spring boot","hibernate","j2ee",
        "jsp","jdbc","maven","gradle","microservices"
    ],

    "DevOps": [
        "docker","kubernetes","aws","azure","linux",
        "ci/cd","terraform","jenkins","ansible","gitlab"
    ],

    "Database": [
        "sql","oracle","mysql","postgresql","db2",
        "pl/sql","sql server","indexing","query optimization"
    ],

    "SQL Developer": [
        "sql","t-sql","pl/sql","stored procedures",
        "views","triggers","etl","query optimization"
    ],

    "ETL Developer": [
        "informatica","ssis","sql","etl pipelines",
        "data warehousing","data migration","unix","shell scripting"
    ],

    "Testing": [
        "manual testing","automation testing","selenium",
        "test cases","jira","regression testing","api testing"
    ],

    "Web Designing": [
        "html","css","javascript","bootstrap","sass",
        "responsive design","ui design","ux design","figma"
    ],

    "React Developer": [
        "react","javascript","html","css","redux",
        "hooks","rest api","nodejs","json"
    ],

    "DotNet Developer": [
        ".net","c#","asp.net","mvc","web api",
        "entity framework","sql server","linq"
    ],

    "SAP Developer": [
        "sap","abap","sap hana","sap fico",
        "sap mm","sap sd","sap bw"
    ],

    "Blockchain": [
        "blockchain","solidity","smart contracts",
        "ethereum","web3","crypto","consensus algorithms"
    ],

    "Network Security Engineer": [
        "network security","firewalls","vpn",
        "ids","ips","cisco","palo alto","linux"
    ],

    "Information Technology": [
        "it support","system administration","networking",
        "windows server","linux","troubleshooting","cloud computing"
    ],

    "Business Analyst": [
        "requirements gathering","data analysis","stakeholder management",
        "business process","uml","sql","documentation","gap analysis"
    ],

    "PMO": [
        "project management","pmo","risk management",
        "budgeting","stakeholder communication","pmi","itil"
    ],

    "Management": [
        "team management","leadership","operations management",
        "strategic planning","decision making","performance management"
    ],

    "Operations Manager": [
        "operations management","process improvement",
        "resource planning","logistics","kpi","cost reduction"
    ],

    "Sales": [
        "sales","lead generation","crm",
        "business development","negotiation","customer acquisition"
    ],

    "Finance": [
        "financial analysis","accounting","budgeting",
        "auditing","taxation","financial reporting","excel"
    ],

    "Accountant": [
        "general ledger","financial reporting","tax preparation",
        "accounts payable","accounts receivable","quickbooks","sap fi"
    ],

    "Banking": [
        "banking operations","credit analysis",
        "risk management","loan processing","customer relations"
    ],

    "Advocate": [
        "legal drafting","litigation","legal research",
        "contract law","criminal law","case analysis"
    ],

    "Human Resources": [
        "recruitment","talent acquisition","payroll",
        "hr operations","employee engagement","onboarding","performance appraisal"
    ],

    "Public Relations": [
        "media relations","corporate communications",
        "content creation","brand management","public speaking","crisis communication"
    ],

    "Digital Media": [
        "digital marketing","social media management",
        "content strategy","seo","analytics","campaign management"
    ],

    "Designing": [
        "graphic design","photoshop","illustrator",
        "ui design","ux design","branding","visual design"
    ],

    "Architecture": [
        "autocad","revit","architectural design",
        "3d modeling","construction drawings","urban planning"
    ],

    "Civil Engineer": [
        "autocad","structural design","site supervision",
        "project planning","quantity surveying","construction management"
    ],

    "Mechanical Engineer": [
        "autocad","solidworks","ansys",
        "mechanical design","maintenance","manufacturing processes"
    ],

    "Electrical Engineering": [
        "electrical design","power systems",
        "plc","autocad","maintenance","troubleshooting"
    ],

    "Automobile": [
        "vehicle maintenance","mechanical repair",
        "diagnostics","automotive systems","engine repair"
    ],

    "Aviation": [
        "aviation safety","maintenance procedures",
        "flight operations","quality assurance","regulatory compliance"
    ],

    "Health and Fitness": [
        "personal training","nutrition","fitness assessment",
        "strength training","injury prevention","wellness coaching"
    ],

    "Agriculture": [
        "crop management","soil analysis",
        "irrigation systems","farm operations","harvesting"
    ],

    "Food and Beverages": [
        "food safety","kitchen operations",
        "inventory management","menu planning","customer service"
    ],

    "Arts": [
        "painting","drawing","visual arts",
        "illustration","sketching","creative design"
    ],

    "Education": [
        "lesson planning","classroom management",
        "curriculum development","student assessment","teaching"
    ],

    "Consultant": [
        "consulting","strategy","process improvement",
        "stakeholder management","business transformation","analysis"
    ],

    "BPO": [
        "customer support","process outsourcing",
        "call center operations","service delivery","client handling"
    ]
}


In [ ]:
COMMON_SKILLS = [
    "python","java","c","c#","c++","sql","pl/sql",
    "html","css","javascript","jquery","jsp","jsf",
    "spring","hibernate","maven","ant","junit","log4j",
    "oracle","mysql","db2","sql server",
    "aws","linux","docker","kubernetes",
    "websphere","weblogic","jboss","tomcat",
    "rest","soap","xml","json",
    "machine learning","nlp","data analysis"
]

In [ ]:
def clean_tokens(text):
    tokens = re.split(r",|•|\||/|\n", text)
    return [
        t.strip().lower()
        for t in tokens
        if 2 < len(t.strip()) < 40
    ]


In [ ]:
def extract_skills_final(text, category=None):
    text_low = text.lower()
    skills = []

    #Skill section
    for match in SKILL_SECTION_PATTERN.findall(text):
        content = match[1]
        skills.extend(clean_tokens(content))

    #Category dictionary
    if category in CATEGORY_SKILLS:
        for s in CATEGORY_SKILLS[category]:
            if s in text_low:
                skills.append(s)

    #Global fallback
    for s in COMMON_SKILLS:
        if s in text_low:
            skills.append(s)

    skills = list(set(skills))
    skills = [s for s in skills if len(s.split()) <= 4]

    if not skills:
        return ["Not Mentioned"]

    return skills


In [ ]:
df_unique['skills'] = df_unique['Text'].apply(extract_skills_final)

In [ ]:
BAD_TOKENS = {
    "ant","rest","work","page","page 2 of 2","summary","references",
    "professional","interests","hobbies","languages","language",
    "skills","highlights","contact","address","resume","cv",
    "linkedin","twitter","instagram","facebook","www","http","https",
    "work history","profile","name","company name", "phone", "tel", "present", "current",
    "reference", "email", "wpm", "born", "date of birth"
}

ALLOWED_SHORT = {"c", "r", "go", "js", "sql", "c++", "c#"}  # tek harfli C'yi bilinçli olarak attık

EMAIL_RE = re.compile(r'\b[\w\.-]+@[\w\.-]+\.\w+\b')
PHONE_RE = re.compile(r'\+?\d[\d\-\s\(\)]{6,}')
URL_RE   = re.compile(r'(https?://|www\.)\S+')
YEAR_RE  = re.compile(r'\b(19|20)\d{2}\b')
NUMERIC_RE = re.compile(r'^[\d\W_]+$')

EMAIL_RE = re.compile(r'\b[\w\.-]+@[\w\.-]+\.\w+\b')
PHONE_RE = re.compile(
    r'(\+?\d{1,3}[\s\-]?)?(\(?\d{2,4}\)?[\s\-]?)?\d{3,4}[\s\-]?\d{3,4}'
)
URL_RE = re.compile(r'(https?://|www\.)\S+')
YEAR_RE = re.compile(r'\b(19|20)\d{2}\b')
# Sadece rakam / sembolden oluşan anlamsız tokenlar
NUMERIC_RE = re.compile(r'^[\d\W_]+$')

# Bullet & özel unicode işaretler
BULLET_RE = re.compile(r'[•⚫▪◦●■◆▶►–—◄]')

# Page, resume footer vs.
PAGE_RE = re.compile(r'\b(page|sayfa)\s*\d+(\s*of\s*\d+)?', re.I)

# Ay + yıl (Jan 2020, January 2021 vb.)
MONTH_YEAR_RE = re.compile(
    r'\b(jan|feb|mar|apr|may|jun|jul|aug|sep|sept|oct|nov|dec)[a-z]*\s+(19|20)\d{2}\b',
    re.I
)
# Tarih aralıkları (2018-2021, 2019 – Present)
DATE_RANGE_RE = re.compile(
    r'\b(19|20)\d{2}\s*[-–—]\s*((19|20)\d{2}|present|current)\b',
    re.I
)
# Reference satırları
REFERENCE_RE = re.compile (r'\breference[s]?\b.*', re.I )
#  Address / lokasyon gürültüsü
ADDRESS_RE = re.compile(
    r'\b(street|st\.|road|rd\.|avenue|ave\.|city|state|zip|postal|blvd|lane|ln)\b',
    re.I
)
SECTION_HEADER_RE = re.compile(
    r'\b(skills?|experience|education|summary|profile|interests|hobbies)\b',
    re.I
)


In [ ]:
import re

def clean_skills(skill_list):
    """
    Cleans a list of extracted skill tokens.
    - Removes bullets/special chars
    - Filters emails/phones/urls/years/numeric-only tokens if corresponding regexes exist
    - Removes very short/very long/noisy tokens
    - Preserves order and deduplicates
    Returns ["Not Mentioned"] if nothing remains.
    """

    if not isinstance(skill_list, list):
        return ["Not Mentioned"]

    cleaned = []
    seen = set()

    for s in skill_list:
        if not isinstance(s, str):
            continue

        t = s.strip()
        # remove bullet-like characters if BULLET_RE exists
        if "BULLET_RE" in globals() and hasattr(BULLET_RE, "sub"):
            t = BULLET_RE.sub(" ", t)

        # basic punctuation trim then normalize
        t = t.strip(".,;:()[]{}\"'").lower()
        # collapse multi-spaces
        t = re.sub(r'\s{2,}', ' ', t).strip()

        # skip empty
        if not t:
            continue

        # helper to safely test regexes only if defined
        def rx_search(name):
            obj = globals().get(name)
            return bool(obj and hasattr(obj, "search") and obj.search(t))

        def rx_fullmatch(name):
            obj = globals().get(name)
            return bool(obj and hasattr(obj, "fullmatch") and obj.fullmatch(t))

        def rx_match(name):
            obj = globals().get(name)
            return bool(obj and hasattr(obj, "match") and obj.match(t))

        # direct blacklist tokens (if BAD_TOKENS exists)
        bad_tokens = globals().get("BAD_TOKENS", set())
        if t in bad_tokens:
            continue

        # regex-based rejects (only if regex defined)
        if rx_search("EMAIL_RE") or rx_search("PHONE_RE") or rx_search("URL_RE") or rx_search("YEAR_RE"):
            continue
        if rx_match("NUMERIC_RE"):
            continue

        # other optional noisy patterns
        if rx_search("PAGE_RE") or rx_search("MONTH_YEAR_RE") or rx_search("DATE_RANGE_RE") or rx_search("REFERENCE_RE") or rx_search("ADDRESS_RE"):
            continue
        if rx_fullmatch("SECTION_HEADER_RE"):
            continue

        # short token handling (ALLOWED_SHORT optional set)
        allowed_short = globals().get("ALLOWED_SHORT", set())
        if len(t) <= 2 and t not in allowed_short:
            continue

        # drop overly long garbage lines
        if len(t) > 70:
            continue

        # final normalization: strip leftover non-alnum ends
        t = re.sub(r'^[^\w]+|[^\w]+$', '', t).strip()
        if not t:
            continue

        # preserve order and dedupe
        if t not in seen:
            cleaned.append(t)
            seen.add(t)

    return cleaned if cleaned else ["Not Mentioned"]


In [ ]:
df_unique["skills"] = df_unique["skills"].apply(clean_skills)

In [ ]:
df_unique.head()

,Category,Text,email,phone,date,experience_entries,experience_total_months_parsed,education,skills
0,Accountant,"﻿________________\r\n\r\nEDUCATION\r\nOMBA - Executive Leadership University of Texas\r\n2016-2018\r\nO Bachelor of Science in Accounting Richland College\r\n2005-2008\r\nTRAINING & CERTIFICATIONS\r\nCertified Management Accountant (CMA)\r\nCertified Financial Modeling and Valuation Analyst\r\nCompliance and Anti-Money Laundering (09/2016) American Institute of Banking\r\nCertified Public Account (CPA)\r\nLean Six Sigma Green Belt Certified\r\nTrade Products Financial Regulations (08/2016) American Institute of Banking\r\nACHIEVEMENTS\r\nSpeaker | Bringing out the Leader within YOU (08/2019)\r\nSuccessfully presented an empowering speech on leadership to a 500+ participants.\r\nSpeaker | Dallas Convention of CPAs (03/2019) Successfully delivered a seminar to 3K+ CPAs and convention guests\r\nTEACHING EXPERIENCE\r\nOnline Teacher - Udemy (2017)\r\nTaught Online Accounting for Non-Accountant Course through Udemy and similar online teaching platforms.\r\n- Developed effective teaching modules, materials, and curriculum for the target students.\r\nTook feedbacks from students to assist in improving the teaching methodology and materials.\r\nPROFESSIONAL MEMBERSHIPS & AFFILIATIONS\r\nAmerican Society of Executives (2018 - Present)\r\nTECHNICAL SKILLS\r\nQuickbooks\r\nERP (SAP & Oracle)\r\nHyperion\r\nLANGUAGES\r\nEnglish\r\nNative\r\nFellow Chartered Accountant (2011 - Present)\r\nMS Office\r\nSQL\r\nIBM Cognos\r\nGerman\r\nSpanish\r\nFrench\r\nFull Professional Proficiency\r\nLimited Working Proficiency\r\nLimited Working Proficiency\r\nINTERESTS\r\nArtificial Intelligence\r\nChess\r\nNnovoresume.com\r\nSailing\r\nCryptocurrencies\r\nPage 2 of 2",None,None,(2011 - Present); 2016-2018; (2018 -; (09/2016); (08/2016); (03/2019); 2005-2008; (2017); (08/2019),"[{'start_dt': 2018-01-01 00:00:00, 'end_dt': 2026-01-01 21:26:25.122165, 'title': None, 'company': None, 'contexts': ['proving the teaching methodology and materials. PROFESSIONAL MEMBERSHIPS & AFFILIATIONS American Society of Executives (2018 - Present) TECHNICAL']}]",97,"[omba - executive leadership university of texas, o bachelor of science in accounting richland college, training & certifications, certified management accountant (cma, certified financial modeling and valuation analyst, compliance and anti-money laundering (09/2016) american institute of banking, certified public account (cpa, lean six sigma green belt certified, trade products financial regulations (08/2016) american institute of banking]","[c, sailing, german, limited working proficiency, erp (sap & oracle, spanish, ms office, sql, full professional proficiency, ibm cognos, cryptocurrencies, english, nnovoresume.com, chess, french, artificial intelligence, oracle, hyperion, native, quickbooks]"
1,Accountant,"﻿________________\r\n\r\nHOWARD GERRARD\r\nAccountant\r\nDeyjob.com, Birmingham\r\n0123 456 7890 info@dayjob.com\r\nin Linkedin/name\r\nWORK EXPERIENCE\r\nCompany name\r\nACCOUNTANT - Location\r\nSUMMARY\r\nHoward is considered an expert at managing a company's finances and looking after its accounts. He is able to perform mathematical calculations with speed as well as accuracy and can be relied upon to provide accurate and trustworthy information about financial transactions. He is confident in his ability to play a strategic role in any business.\r\nEDUCATION\r\nUniversity\r\nCourse details\r\n2010-2013\r\n2016-Present\r\nResponsible for ensuring the smooth running of the accounts department. Advising and updating the Director on irregularities or concerns about invoices & financial matters.\r\n• Keeping financial control records for expenditures, allotments and receipts etc.\r\n• Verifying financial documents for validity, accuracy and completeness.\r\n• Managing the receipts and deposits of cash and bank transactions.\r\n• Ensuring that all invoices and staff reimbursements are paid accur

In [ ]:
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12175 entries, 0 to 13388
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Category                        12175 non-null  object
 1   Text                            12175 non-null  object
 2   email                           10615 non-null  object
 3   phone                           8516 non-null   object
 4   date                            11924 non-null  object
 5   experience_entries              12175 non-null  object
 6   experience_total_months_parsed  12175 non-null  int64 
 7   education                       12175 non-null  object
 8   skills                          12175 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.2+ MB


In [ ]:
import os
os.makedirs("/content/drive/MyDrive/cv_matching", exist_ok=True)

In [ ]:
save_path = "/content/drive/MyDrive/cv_matching/df_unique.csv"
df_unique.to_csv(save_path, index=False)

print("Kaydedildi:", save_path)


Kaydedildi: /content/drive/MyDrive/cv_matching/df_unique.csv


İŞ İLANINA UYGUN CV BULMA


In [ ]:
!pip install gradio
import gradio as gr


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

DATA_PATH = "/content/drive/MyDrive/cv_matching/df_unique.csv"
df_unique = pd.read_csv(DATA_PATH)


In [ ]:
df_unique["skills_list"] = df_unique["skills"].apply(
    lambda x: [s.strip().lower() for s in x.split(",")] if isinstance(x, str) else []
)


In [ ]:
df_unique["experience_years"] = df_unique["experience_total_months_parsed"] / 12


In [ ]:
def normalize_education(text):
    if not isinstance(text, str):
        return None

    t = text.lower()

    if "phd" in t or "ph.d" in t or "doctor" in t:
        return "PhD"
    elif "master" in t or "msc" in t or "m.sc" in t or "yüksek" in t:
        return "Master"
    elif "bachelor" in t or "b.sc" in t or "bsc" in t or "lisans" in t:
        return "Bachelor"
    else:
        return None


In [ ]:
df_unique["education_norm"] = df_unique["education"].apply(normalize_education)


In [ ]:
df_unique["education_norm"].value_counts(dropna=False)


,count
education_norm,
None,5207
Bachelor,4056
Master,2772
PhD,140


In [ ]:
education_choices = ["No preference", "Bachelor", "Master", "PhD"]


In [ ]:
def make_clickable_email(email):
    if isinstance(email, str) and "@" in email:
        return f'<a href="mailto:{email}">{email}</a>'
    return ""


In [ ]:
gr.Dropdown(
    choices=education_choices,
    value="No preference",
    label="Minimum Education"
)

In [ ]:
def score_cv(row, user):
    score = 0

    # Category
    if row["Category"] == user["category"]:
        score += 4

    # Skills (ANA KRİTER)
    overlap = set(row["skills_list"]) & set(user["skills"])
    score += len(overlap) * 3

    # Experience (ANA KRİTER)
    if row["experience_years"] >= user["min_experience"]:
        score += 3

    # Education (OPSİYONEL)
    if user["education"] != "No preference":
        edu_rank = {"Bachelor": 1, "Master": 2, "PhD": 3}
        if (
            row["education_norm"] in edu_rank and
            edu_rank[row["education_norm"]] >= edu_rank[user["education"]]
        ):
            score += 2

    return score


In [ ]:
def match_cvs(category, skills, education, min_experience):
    user = {
        "category": category,
        "skills": [s.strip().lower() for s in skills.split(",")],
        "education": education,
        "min_experience": min_experience
    }

    df = df_unique.copy()
    df["score"] = df.apply(lambda x: score_cv(x, user), axis=1)

    top10 = df.sort_values("score", ascending=False).head(10)

    top10["email"] = top10["email"].apply(make_clickable_email)

    return top10[
    [
        "Category",
        "email",
        "education_norm",
        "experience_years",
        "skills",
        "score"
    ]
    ]


In [ ]:
gr.Interface(
    fn=match_cvs,
    inputs=[
        gr.Dropdown(
            choices=sorted(df_unique["Category"].unique().tolist()),
            label="Job Category"
        ),
        gr.Textbox(
            label="Required Skills (comma separated)",
            placeholder="python, sql, machine learning"
        ),
        gr.Dropdown(
            choices=["No preference", "Bachelor", "Master", "PhD"],
            value="No preference",
            label="Minimum Education"
        ),
        gr.Number(
            label="Minimum Experience (Years)",
            value=2
        )
    ],
    outputs=gr.Dataframe(
        label="Top 10 Matching CVs",
        datatype="html"
    ),
    title="HR CV Matching System",
    description="Enter job requirements to retrieve the most relevant CVs"
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://49b7425ce3d2a466f7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
